<div align="center">
 <img src="http://www.di.uoa.gr/themes/corporate_lite/logo_en.png" title="Department of Informatics and Telecommunications - University of Athens" align="center" /> 
</div>

<br>

---

<div align="center"> 
  <font size="3"><b>Bachelor Thesis</b> </font>
</div>
<br>
<div align="center"> 
  <font size="5">
      <b>WinnER: A Winner-Take-All Hashing-Based Unsupervised Model <br>for Entity Resolution Problems<br></b> 
    </font>
     <hr>
     <font size="4">
        Study on CDDB data set 
    </font>
</div>

---

<div align="center"> 
    <font size="4">
         <b>Konstantinos Nikoletos</b>, BS Student at NKUA 
     </font>
</div>
<div align="center"> 
    <font size="4">
     <b> Alex Delis</b>,  Professor NKUA <br> 
     <b> Vassilios Verykios</b>, Professor Hellenic Open University
    </font>
</div>
<br>
<div align="center"> 
    <font size="2">Athens</font>
</div>
<div align="center"> 
    <font size="2">January 2021 - Ongoing</font>
</div>


---

# __Code injection__

## Import of libraries

In [24]:
import pandas as pd
import numpy as np
import collections
import editdistance
import string
import sklearn
import pandas_read_xml as pdx
import time
import warnings
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import math
import os
import scipy.special as special
import igraph
import networkx as nx
import itertools 
import sklearn
import plotly.express as px
import logging
import sys
import optuna
import plotly

from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_slice
from tqdm.notebook import tqdm as tqdm
from scipy.spatial.distance import directed_hausdorff,hamming
from scipy.stats._stats import _kendall_dis
from scipy.stats import spearmanr,kendalltau,pearsonr,kruskal,mannwhitneyu
from sklearn.metrics.pairwise import cosine_similarity
from nltk.metrics.distance import jaro_similarity,jaro_winkler_similarity,jaccard_distance
from sklearn.metrics import jaccard_score,accuracy_score,auc,f1_score,recall_score,precision_score,classification_report
from scipy.sparse import csr_matrix
from scipy import sparse
from scipy import stats 
from scipy.spatial.distance import euclidean,hamming,jaccard
from matplotlib.patches import Rectangle
from sklearn.metrics import ndcg_score
from datetime import datetime
from sklearn.decomposition import PCA 
from numpy.linalg import svd

# plt.style.use('seaborn-white') # plot style

Automated refresh on the notebook

In [25]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Main model dependency

In [26]:
from model import WinnER, customClassificationReport, report

## Utilities

In [27]:
from utils.mmd2 import MMD2, myMMD2
from plot.heatmap import myHeatmap
from plot.confusion_matrix import create_ConfusionMatrix
from plot.dimension_reduction import SpaceVisualization2D, SpaceVisualization3D, SpaceVisualizationEmbeddings2D, SpaceVisualizationEmbeddings3D
from plot.heatmap import similarityProbsHeatMap, similarityProbsHeatMapWithClusters

### Dataset utils

In [28]:
from datasets.common.create_dataset import createDataset, createTrueLabels
from utils.grid_search import GridSearch

### Custom metrics

In [29]:
from utils.metrics import spearman_footrule_distance, kendall_top_k, rbo


# __Dataset injection__

## Load data from Drive in Colab

In [30]:
# # Opening data file
# import io
# from google.colab import drive

# drive.mount('/content/drive',force_remount=True)

# fp = r"/content/drive/My Drive/ERinDS/CORA.xml"
# fp_gold = r"/content/drive/My Drive/ERinDS/cora_gold.csv"

## Load data from disk for Jupyter

In [35]:
CDDB_groundTruth = pd.read_csv(os.path.abspath("../data/cddbIdDuplicates.csv"),sep='/00000',engine='python',header=None,names=['id1','id2'])
CDDB = pd.read_csv(os.path.abspath("../data/cddbProfiles.csv"),sep='/00000',engine='python')
CDDB['Entity Id'] = CDDB.index
CDDB

,Entity Id|artist,category,cdextra,genre,title,track01,track02,track03,track04,track05,...,track93,track94,track95,track96,track97,track98,track99,year,Unnamed: 105,Entity Id
0,pink floyd,data,ID3G: 254,Data,the wall disc 1,in the flesh,the thin ice,another brick in the wall part 1,the happiest days of our lives,another brick in the wall part 2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Carlos Santana,data,NaN,NaN,Carlos,(Da Le) Yaleo,Love Of My Life,Put You Love Lights On,Smooth,Do You Like The Way?,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,Frans Bauer,data,NaN,Other,'n ons geluk,'n ONS geluk,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004,NaN,2
3,2Pac,data,YEAR: 1998 ID3G: 15,Rap,Greatest Hits (CD1),Keep Ya Head Up,2 Of Amerikaz Most Wanted (Ft. Snoop Dogg),Temptations,God Bless The Dead,Hail Mary,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1998,NaN,3
4,̤��,data,NaN,NaN,̤��,�ȥ�å� 1,�ȥ�å� 2,�ȥ�å� 3,�ȥ�å� 4,�ȥ�å� 5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9758,Various,soundtrack,NaN,NaN,Warriors of Virtue,You Can Fly,A Beautiful Morning,Forces of Nature,Inside of You,Tennessee Plates,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9758
9759,James Patterson,soundtrack,NaN,Soundtrack,Pop Goes the Weasel - CD 4,4.01,4.02,4.03,4.04,4.05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,NaN,9759
9760,Colin Baker & Nicholas Courtney,soundtrack,Published by Big Finish Productions.\nhttp://w...,Soundtrack,The Spectre of Lanyon Moor,Episode 0,Episode 1 Part 1,Episode 1 Part 2,Episode 1 Part 3,Episode 1 Part 4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,NaN,9760
9761,Studio Cutz,soundtrack,ID3G: 24,Soundtrack,Volume 25,All Access 3:12,All Access 3:12 alt,All Access :60,All Access :30,All Access :11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9761


In [32]:
CDDB_groundTruth.sort_values('id1')

,id1,id2
176,0,220
27,1,394
148,2,9
64,3,292
182,4,410
...,...,...
193,460,466
239,461,469
39,463,464
139,465,470


# __Dataset attribute analysis__

In [33]:
DATASET_NAME = 'CDDB'

## Data set shuffle (Optional)

In [34]:
dataset_shuffled = CDDB.sample(frac=1).reset_index(drop=True)
# dataset_shuffled

## Pre-process

In [ ]:
fields = list(CDDB.columns)

# All
data, true_matrix = createDataset(CDDB, CDDB_groundTruth, fields, 'Entity Id')

__Clique construction for visualization purposes__

In [36]:
labels_groundTruth, numOfObjWithoutDups, groups = createTrueLabels(CDDB['Entity Id'].tolist(), CDDB_groundTruth)            
data_length = [ len(x) for x in data ]

## Attributes

In [ ]:
specs = {
    'Objects without any duplicates' : CDDB.shape[0] - sum([len(x) for x in groups]), 
    'Objects with at least one duplicate' : sum([len(x) for x in groups]),
    'Total number of objects' : CDDB.shape[0], 
    'Mean size of clusters' : np.average([len(x) for x in groups]),
    'Number of ER clusters' : len(groups),
    'Dataset size' : len(data), 
    'Average length' : np.mean(data_length), 
    'Min length' : min(data_length), 
    'Max length' : max(data_length),
    'Median length' : np.median(data_length)
}
specsCDDBDf = pd.DataFrame(list(specs.items()), columns=['Spec','#count'])
specsCDDBDf[['#count']] = specsCDDBDf[['#count']].astype(int)
specsCDDBDf

In [ ]:
df = pd.DataFrame({"Record index":range(0,len(data_length),1),"String length":data_length})
fig = px.bar(df, x= "Record index", y="String length", title= DATASET_NAME + " Data set string length bar plot",template='plotly_white')
fig.update_traces(marker_color='darkblue', opacity=1.0)
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
fig = plt.figure(figsize=(15,5))
plt.bar(range(0,len(data_length),1),data_length)
plt.xlabel("Record index")
plt.ylabel("String length")
plt.title(DATASET_NAME + " Data set string length bar plot")
fig.savefig("strlenncddb.png")
plt.show()

# Optuna search

In [ ]:
SpaceVisualization2D(winnER.Embeddings, winnER.prototypeArray, withPlotly=False)

In [ ]:
SpaceVisualization2D(winnER.Embeddings, winnER.prototypeArray)

In [ ]:
SpaceVisualization2D(winnER.Embeddings, winnER.prototypeArray, decompositionMenthod='MDS')

In [ ]:
SpaceVisualization3D(winnER.Embeddings, winnER.prototypeArray)

In [ ]:
SpaceVisualization3D(winnER.rankedVectors,winnER.prototypeArray, decompositionMenthod='MDS')

In [ ]:
SpaceVisualization3D(winnER.rankedVectors, winnER.prototypeArray, withPlotly = False, decompositionMenthod='MDS')

In [ ]:
SpaceVisualization3D(winnER.rankedVectors, winnER.prototypeArray, withPlotly = True, decompositionMenthod='MDS')

In [ ]:
SpaceVisualizationEmbeddings2D(
    winnER.Embeddings, labels_groundTruth, withPlotly=True, decompositionMenthod='PCA'
)

In [ ]:
SpaceVisualizationEmbeddings3D(
    winnER.rankedVectors, labels_groundTruth, withPlotly=False, decompositionMenthod='PCA'
)

In [ ]:
SpaceVisualizationEmbeddings3D(
    winnER.rankedVectors, labels_groundTruth, withPlotly=True, decompositionMenthod='PCA'
)

In [ ]:
SpaceVisualizationEmbeddings3D(
    winnER.rankedVectors, labels_groundTruth, withPlotly=True, decompositionMenthod='MDS'
)

__MMD error between distribution of Prototypes and whole Dataset__

In [ ]:
myMMD2(winnER)

## Euclid-Jaccard with 3-grams and char-tokenization

In [ ]:
euclid_jaccard_char_3 = optuna.create_study(
    directions = ["maximize"],
    study_name = "euclid_jaccard_char_3_CDDB", 
    storage    = "sqlite:///optuna_tuning/WinnER_Experiments_v3.db", 
    load_if_exists = True
)

Trials dataframe (Optuna default)

In [ ]:
euclid_jaccard_char_3.trials_dataframe(attrs=("number", "value", "params", "state"))

In [ ]:
plot_optimization_history(euclid_jaccard_char_3)

In [ ]:
# plot_intermediate_values(euclid_jaccard_char_3)

In [ ]:
plot_parallel_coordinate(euclid_jaccard_char_3)

In [ ]:
plot_parallel_coordinate(euclid_jaccard_char_3, params=["number_of_permutations", "similarity_threshold"])

In [ ]:
plot_contour(euclid_jaccard_char_3)

In [ ]:
plot_contour(euclid_jaccard_char_3, params=["number_of_permutations", "similarity_threshold"])

In [ ]:
plot_slice(euclid_jaccard_char_3,  params=["number_of_permutations", "similarity_threshold", "max_dissimilarity_distance", "window_size", "max_num_of_clusters"])

In [ ]:
plot_param_importances(euclid_jaccard_char_3)

In [ ]:
plot_edf(euclid_jaccard_char_3)

In [ ]:
optuna.visualization.plot_param_importances(
    euclid_jaccard_char_3, target=lambda t: t.duration.total_seconds(), target_name="duration"
)

### Study statistics

In [ ]:
euclid_jaccard_char_3_DF = pd.read_csv(
    os.path.abspath("./optuna_tuning/csv/euclid_jaccard_char_3_CDDB.csv")
)
print(euclid_jaccard_char_3_DF.columns)

In [ ]:
ltable = euclid_jaccard_char_3_DF[['trial_id','Recall','F1','Precision','Accuracy', 'Time']].sort_values(by=['F1','Recall','F1'], ascending=False).head(10)
ltable = ltable.sort_values(by=['Recall','F1'], ascending=False)
ltable['Time'] = ltable['Time'].astype('datetime64[s]').dt.strftime("%M:%S")
print(ltable.to_latex(index=False))



In [ ]:
fig = px.parallel_coordinates(
    euclid_jaccard_char_3_DF[[ 'diffObjectsComparedSuccess',
       'sameObjectsCompared', 'sameObjectsComparedSuccess', 'selectedNumOfPrototypes', 'number_of_permutations', 'num_of_comparisons', 'Recall']], color="Recall", 
    # labels={"num_of_comparisons": "num_of_comparisons", "diffObjectsComparedSuccess": "diffObjectsComparedSuccess"},
    color_continuous_scale=px.colors.sequential.RdBu, 
    color_continuous_midpoint=2
)
fig.show()

In [ ]:
fig = px.parallel_coordinates(
    euclid_jaccard_char_3_DF[[ 'max_dissimilarity_distance', 'max_num_of_clusters', 'selectedNumOfPrototypes', 'Recall', 'Accuracy']], color="Recall", 
    # labels={"num_of_comparisons": "num_of_comparisons", "diffObjectsComparedSuccess": "diffObjectsComparedSuccess"},
    color_continuous_scale=px.colors.sequential.RdBu, 
    # color_continuous_midpoint=2
)
fig.show()

In [ ]:
# fig = px.parallel_categories(
#     euclid_jaccard_char_3_DF[[ 'diffObjectsComparedSuccess', 'sameObjectsCompared', 'sameObjectsComparedSuccess', 'selectedNumOfPrototypes', 'number_of_permutations', 'num_of_comparisons', 'Recall', 'trial_id']], 
#     color="Recall",
#     color_continuous_scale=px.colors.sequential.Inferno
# )
# fig.show()

In [ ]:
fig = px.scatter(
    euclid_jaccard_char_3_DF[['F1', 'Time', 'diffObjectsComparedSuccess', 'sameObjectsCompared', 'sameObjectsComparedSuccess', 'selectedNumOfPrototypes', 'number_of_permutations', 'num_of_comparisons', 'Recall', 'trial_id']], 
    x="diffObjectsComparedSuccess", 
    y="sameObjectsComparedSuccess", 
    size="num_of_comparisons", 
    color="Recall",
    # hover_name="F1", 
    log_x=True, 
    color_continuous_scale=px.colors.sequential.RdBu_r, 
    size_max=10,
    template='plotly_white',
    labels={
         "diffObjectsComparedSuccess": "Different papers compared with success",
         "sameObjectsComparedSuccess": "Same papers compared with success",
     }
)
fig.show()

In [ ]:
plot_df = euclid_jaccard_char_3_DF[['F1', 'Time', 'diffObjectsComparedSuccess', 'sameObjectsCompared', 'sameObjectsComparedSuccess', 'selectedNumOfPrototypes', 'number_of_permutations', 'num_of_comparisons', 'Recall', 'trial_id']].sort_values('Recall', ascending=True).tail(10)

px.scatter(
    plot_df, 
    x="diffObjectsComparedSuccess", 
    y="sameObjectsComparedSuccess", 
    # size="num_of_comparisons", 
    # color="Time",
    # hover_name="F1", 
    animation_frame="Recall", 
    animation_group="number_of_permutations",
    # facet_col="Recall",
    log_x=True, 
    template='plotly_white'
    # size_max=10, 
    # range_x=[1000,10000000], 
    # range_y=[10,100]
)
# fig.show()

In [ ]:
fig = px.parallel_coordinates(
    euclid_jaccard_char_3_DF[[ 'window_size', 'number_of_permutations', 'averageBucketSize','similarityTime', 'Recall', 'Accuracy']], color="Recall", 
    # labels={"num_of_comparisons": "num_of_comparisons", "diffObjectsComparedSuccess": "diffObjectsComparedSuccess"},
    color_continuous_scale=px.colors.sequential.RdBu, 
    # color_continuous_midpoint=2
)
fig.show()

### Best model

In [ ]:
# euclid_jaccard_char_3.best_trial.number
# euclid_jaccard_char_3_DF[]
best_params = euclid_jaccard_char_3_DF[euclid_jaccard_char_3_DF['trial_id'] == 12].to_dict('records')[0]

In [ ]:
best_params

In [ ]:
winnER = WinnER(verbose_level=1)
winnER.set_params(best_params)
winnER.hackForDebug(labels_groundTruth, true_matrix)
winnER.fit(data)
winnER.evaluate(winnER.mapping_matrix, true_matrix)

### Space Visualization

In [ ]:
SpaceVisualization2D(winnER.Embeddings, winnER.prototypeArray)

In [ ]:
SpaceVisualization2D(winnER.Embeddings, winnER.prototypeArray, decompositionMenthod='MDS')

In [ ]:
SpaceVisualization3D(winnER.Embeddings, winnER.prototypeArray)

In [ ]:
SpaceVisualization3D(winnER.rankedVectors,winnER.prototypeArray, decompositionMenthod='MDS')

In [ ]:
SpaceVisualization3D(winnER.rankedVectors, winnER.prototypeArray, withPlotly = True, decompositionMenthod='MDS')

In [ ]:
SpaceVisualizationEmbeddings2D(
    winnER.Embeddings, labels_groundTruth, withPlotly=True, decompositionMenthod='PCA'
)

In [ ]:
SpaceVisualizationEmbeddings3D(
    winnER.rankedVectors, labels_groundTruth, withPlotly=True, decompositionMenthod='PCA'
)

In [ ]:
SpaceVisualizationEmbeddings3D(
    winnER.rankedVectors, labels_groundTruth, withPlotly=True, decompositionMenthod='MDS'
)

__MMD error between distribution of Prototypes and whole Dataset__

In [ ]:
myMMD2(winnER)